In [ ]:
# downloading the dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-23 11:16:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.61MB/s    in 0.7s    

2025-06-23 11:16:18 (1.61 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# reading the file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [4]:
# taking a look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("Vocab Size: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab Size:  65


In [6]:
# create a mapping from chars to ints
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encode takes a str and returns list of int
decode = lambda l: ''.join([itos[i] for i in l]) # decode takes list of int and converts it to str

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# lets now split the data into train and val
n = int(0.9 * len(data)) # 90% train data and 10% val data
train_data = data[:n]
val_data = data[n:]

In [9]:
# we will send chunks of data into the transformer and not the entire text
block_size = 8
train_data[:block_size+1] # block_size + 1 becuase it 9 characters have 8 examples 
                          # when given 18 model will predict 47, and when given [18, 47] model will predict 56 and so on..

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
# the 8 examples are
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{t+1}: when input is {context} the target is {target}")

1: when input is tensor([18]) the target is 47
2: when input is tensor([18, 47]) the target is 56
3: when input is tensor([18, 47, 56]) the target is 57
4: when input is tensor([18, 47, 56, 57]) the target is 58
5: when input is tensor([18, 47, 56, 57, 58]) the target is 1
6: when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
7: when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
8: when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [11]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [12]:
# to check what is input and what is output
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when context is {context.tolist()} the target is {target}")

when context is [57] the target is 1
when context is [57, 1] the target is 46
when context is [57, 1, 46] the target is 47
when context is [57, 1, 46, 47] the target is 57
when context is [57, 1, 46, 47, 57] the target is 1
when context is [57, 1, 46, 47, 57, 1] the target is 50
when context is [57, 1, 46, 47, 57, 1, 50] the target is 53
when context is [57, 1, 46, 47, 57, 1, 50, 53] the target is 60
when context is [1] the target is 58
when context is [1, 58] the target is 46
when context is [1, 58, 46] the target is 43
when context is [1, 58, 46, 43] the target is 56
when context is [1, 58, 46, 43, 56] the target is 43
when context is [1, 58, 46, 43, 56, 43] the target is 1
when context is [1, 58, 46, 43, 56, 43, 1] the target is 41
when context is [1, 58, 46, 43, 56, 43, 1, 41] the target is 39
when context is [17] the target is 26
when context is [17, 26] the target is 15
when context is [17, 26, 15] the target is 17
when context is [17, 26, 15, 17] the target is 10
when context is

In [13]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits from the next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(self, idx, targets=None):
        # idx and target are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # B,T,C -> (Batch,Training,Vocab_Size)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, target=targets)
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [14]:
m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

# testing the generate func
test_idx = torch.zeros((1,1), dtype=torch.long)
generate_op = m.generate(test_idx, max_new_tokens=100)[0].tolist()
print(generate_op)
print(decode(generate_op))

torch.Size([32, 65]) tensor(4.8865, grad_fn=<NllLossBackward0>)
[0, 53, 3, 6, 55, 4, 21, 35, 55, 35, 4, 62, 58, 15, 48, 39, 14, 12, 47, 48, 4, 40, 37, 30, 19, 49, 18, 12, 40, 11, 1, 44, 1, 6, 15, 40, 61, 46, 58, 17, 30, 15, 21, 44, 59, 35, 56, 6, 16, 64, 22, 17, 30, 48, 46, 24, 50, 34, 39, 18, 4, 17, 48, 44, 44, 28, 20, 16, 18, 41, 26, 53, 19, 21, 19, 5, 4, 3, 55, 36, 47, 57, 35, 32, 49, 22, 28, 61, 0, 1, 6, 40, 1, 36, 45, 62, 12, 16, 9, 57, 48]

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [15]:
# training loop
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
epochs = 10000
for epoch in range(epochs):
    # get a sample batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 4.688536643981934
Epoch: 2, Loss: 4.7616071701049805
Epoch: 3, Loss: 4.763946056365967
Epoch: 4, Loss: 4.746903419494629
Epoch: 5, Loss: 4.848341941833496
Epoch: 6, Loss: 4.714447498321533
Epoch: 7, Loss: 4.662259578704834
Epoch: 8, Loss: 4.643644332885742
Epoch: 9, Loss: 4.838624954223633
Epoch: 10, Loss: 4.68344783782959
Epoch: 11, Loss: 4.663377285003662
Epoch: 12, Loss: 4.749584197998047
Epoch: 13, Loss: 4.755120277404785
Epoch: 14, Loss: 4.735619068145752
Epoch: 15, Loss: 4.7752556800842285
Epoch: 16, Loss: 4.778011798858643
Epoch: 17, Loss: 4.764355659484863
Epoch: 18, Loss: 4.788415908813477
Epoch: 19, Loss: 4.859102249145508
Epoch: 20, Loss: 4.7950005531311035
Epoch: 21, Loss: 4.720877647399902
Epoch: 22, Loss: 4.720705509185791
Epoch: 23, Loss: 4.727301597595215
Epoch: 24, Loss: 4.615210056304932
Epoch: 25, Loss: 4.7232346534729
Epoch: 26, Loss: 4.696660995483398
Epoch: 27, Loss: 4.700191497802734
Epoch: 28, Loss: 4.638886451721191
Epoch: 29, Loss: 4.6715307235

In [16]:
# testing the generate func
test_idx = torch.zeros((1,1), dtype=torch.long)
generate_op = m.generate(test_idx, max_new_tokens=100)[0].tolist()
print(decode(generate_op))


QUDUThe chas.
F lisen tabr:
LI mus nk,
A: al l ayo cenghe's therinvar,
TEsen ithawaneit at islinerai


### Mathematical Trick In Self-Attention

In [17]:
torch.manual_seed(42)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# we want x[b,t] = mean_{i<=t} x[b,i] as we want current i to interact with just prev i's
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

x[0], xbow[0] # in xbow fst row is avg of 1 row (vertically)
              #         sec row is avg of 2 rows (vertically) and so on..

(tensor([[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [ 0.6784, -1.2345],
         [-0.0431, -1.6047],
         [-0.7521,  1.6487],
         [-0.3925, -1.4036],
         [-0.7279, -0.5594],
         [-0.7688,  0.7624]]),
 tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]]))

In [19]:
# the above method is very inefficient we can achieve this with the help of matrix multiplications

a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1,keepdim=True)

b = torch.randint(0,10,(3,2)).float()

c = a @ b

print("a")
print(a)
print("--")

print("b")
print(b)
print("--")

print("c")
print(c)

a
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b
tensor([[0., 1.],
        [3., 0.],
        [1., 1.]])
--
c
tensor([[0.0000, 1.0000],
        [1.5000, 0.5000],
        [1.3333, 0.6667]])


In [20]:
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei,1,keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [21]:
xbow2 = wei @ x # wei -> (TxT) and x -> (BxTxC)
                # since the shapes does not match pytorch adds batch dim on its own so wei become (BxTxT)
                # So for every batch dim b we mul a matrix of txt to txc and get output of txc
                # hence the output will be BxTxC

# to see if xbow and xbow2 is same 
torch.allclose(xbow, xbow2)

True

In [25]:
# version 3 of achieving same thing: uses softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [26]:
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

### Self-Attention

In [32]:
torch.manual_seed(42)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # BxTx16
q = query(x) # BxTx16
wei = q @ k.transpose(-2,-1) # (BxTx16) . (Bx16xT) -> (BxTxT)

wei[0] # now wei is diff for eatch batch and not a constant

tensor([[-0.3332, -1.1723, -1.0216, -0.0545, -1.0950,  0.2735,  0.1340, -0.8490],
        [-0.6597,  0.7869, -1.2725,  1.6851,  0.1159,  0.5450,  0.2356, -0.1962],
        [ 0.3630, -1.5219,  0.7821, -1.7215, -0.3494,  0.2884, -0.1021, -1.4271],
        [-0.1001,  0.8649, -0.0335,  1.0221, -0.1350, -0.3078,  0.1440, -0.3019],
        [ 0.0136, -1.6202, -1.9888, -0.3327, -1.2506, -0.8928, -2.2674,  3.0561],
        [-0.5833,  1.2025, -0.3281,  0.9147,  0.9809, -0.4859,  1.7589,  0.1650],
        [ 1.1351, -1.9940,  1.5545, -1.8037, -0.5062, -2.6109, -1.0739,  1.6430],
        [-1.2784, -0.4554, -1.4118,  0.6392, -0.5780,  1.9291,  1.6689,  0.1103]],
       grad_fn=<SelectBackward0>)

In [33]:
# but we still don't want the curr token interacting with the next token so we put -inf where 0 in tril
tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T)) # now this line is not needed
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1) # and then we normalize it with softmax

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

Notes: 
* Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode token.
* Each example of batch dimension if of course processed completely independently and never 'talk' to each other.
* In an 'encoder' attention block just delete the single line that does masking with tril , allowing all tokens to communicate. This block here is called a 'decoder' attention block because it has triangular masking, and is usually used in autoregressive settings, like language modelling.
* 'self-attention' just means that the keys and values are produced from the same source as queries. In 'cross-attention', the queries will get produced form x, but the keys and values come from some other, external source (eg: and encoder module)
* 'scaled attention' additionally divides wei by 1/sqrt(head_size). This makes it so when input Q, K are unit variance, wei will be unit variance too and softmax will stay diffuse and not saturate too much, illustration below

In [40]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) 

print(f"Key Variance: {k.var():.4f}, Query Variance: {q.var():.4f}, Weight Variance: {wei.var():.4f}")

Key Variance: 0.9372, Query Variance: 1.0881, Weight Variance: 16.7803


In [ ]:
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # scaled attention
print(f"Key Variance: {k.var():.4f}, Query Variance: {q.var():.4f}, Weight Variance: {wei.var():.4f}")

# when we didn't used scaled attention we got wei variance as 16.7 (always be roughly the size of head_size)
# when we apply scaled attn we got wei variance around 1
# Now this is imp because if we didn't use scale then valeus in wei are bigger and that wei after going in softmax func 
# so what will happen is the max value will contribute more in softmax, we don't want that so each no should be normalized.

Key Variance: 0.9372, Query Variance: 1.0881, Weight Variance: 1.0488


In [42]:
class LayerNorm:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)                  # batch mean
        xvar = x.var(1, keepdim=True)                    # batch var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize unit var
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

In [43]:
# testing layer norm
torch.manual_seed(42)
module = LayerNorm(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean and std of one feature across all batch

(tensor(-0.1428), tensor(1.0629))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean and std of a single input from the batch, of its features

(tensor(9.5367e-09), tensor(1.0000))